# 1. Enable GPU within the Colab notebook
Enabling GPU will make YOLO process detections over 100 faster than CPU.

To enable GPU go to Runtime->Change runtime type->Hardware Accelerator->GPU.

To cross-check whether the GPU is enabled, run the next cell.

In [ ]:
!nvidia-smi


Sun Aug  9 09:20:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    73W / 149W |    134MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 2. Connecting to Google Drive 
In the next cell, we will mount our google drive on the runtime so we can write and read files from there.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ln -s /content/drive/My\ Drive/ /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# 3. Cloning and Building Darknet

In [ ]:
# Clone the repository
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14135, done.
remote: Total 14135 (delta 0), reused 0 (delta 0), pack-reused 14135
Receiving objects: 100% (14135/14135), 12.74 MiB | 6.73 MiB/s, done.
Resolving deltas: 100% (9608/9608), done.


In [ ]:
# Configure makefile to have OPENCV, GPU and CUDNN enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

/content/darknet


In [ ]:
# Compile
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

# Data Preparation

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

In [ ]:
# Create .names and .data files
!echo -e 'Wearing Mask\nWearring Mask Not Properly\nNot Wearing Mask' > data/obj.names
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/Face_Mask_Detection' > data/obj.data

In [ ]:
# Save yolov3_training.cfg and obj.names files in Google drive
!cp cfg/yolov3_training.cfg /mydrive/Face_Mask_Detection/yolov3_testing.cfg
!cp data/obj.names /mydrive/Face_Mask_Detection/classes.txt

In [ ]:
# Creating a folder and unzip image dataset
!mkdir data/obj
!unzip /mydrive/Face_Mask_Detection/Dataset2.zip -d data/obj

Archive:  /mydrive/Face_Mask_Detection/Dataset2.zip
   creating: data/obj/Dataset2/
  inflating: data/obj/Dataset2/Mask_1.jpg  
  inflating: data/obj/Dataset2/Mask_1.txt  
  inflating: data/obj/Dataset2/Mask_10.jpg  
  inflating: data/obj/Dataset2/Mask_10.txt  
  inflating: data/obj/Dataset2/Mask_100.jpg  
  inflating: data/obj/Dataset2/Mask_100.txt  
  inflating: data/obj/Dataset2/Mask_101.jpg  
  inflating: data/obj/Dataset2/Mask_101.txt  
  inflating: data/obj/Dataset2/Mask_102.jpg  
  inflating: data/obj/Dataset2/Mask_102.txt  
  inflating: data/obj/Dataset2/Mask_103.jpg  
  inflating: data/obj/Dataset2/Mask_103.txt  
  inflating: data/obj/Dataset2/Mask_104.jpg  
  inflating: data/obj/Dataset2/Mask_104.txt  
  inflating: data/obj/Dataset2/Mask_105.jpg  
  inflating: data/obj/Dataset2/Mask_105.txt  
  inflating: data/obj/Dataset2/Mask_106.jpg  
  inflating: data/obj/Dataset2/Mask_106.txt  
  inflating: data/obj/Dataset2/Mask_107.jpg  
  inflating: data/obj/Dataset2/Mask_107.txt  
  

In [ ]:
# Creating train.txt file
import glob
images_list = glob.glob("data/obj/Dataset2/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

# 4. Download pre-trained weights for the convolutional layers

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-08-04 10:02:50--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   227KB/s    in 9m 31s  

2020-08-04 10:12:23 (278 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



# 5. Training the Face Mask Detector
In the next cell, the training will begin and the weights will be saved in Google drive.

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/Face_Mask_Detection/yolov3_training_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.815710, GIOU: 0.808581), Class: 0.999576, Obj: 0.905328, No Obj: 0.002024, .5R: 1.000000, .75R: 0.818182, count: 11, class_loss = 0.348472, iou_loss = 0.308305, total_loss = 0.656777 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.870215, GIOU: 0.869147), Class: 0.985544, Obj: 0.800538, No Obj: 0.000154, .5R: 1.000000, .75R: 1.000000, count: 4, class_loss = 0.122073, iou_loss = 0.069367, total_loss = 0.191439 
 total_bbox = 623831, rewritten_bbox = 0.031258 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.813627, GIOU: 0.805009), Class: 0.996924, Obj: 0.952909, No Obj: 0.001904, .5R: 1.000000, .75R: 0.750000, count: 4, class_loss = 0.017468, iou_loss = 0.105477, total_loss = 0.122945 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.876429, GIOU: 0.875523), Class: 0.999940, Obj: 0.997519, No Obj